# Requirements

In [ ]:
!pip install -q langchain==0.0.208 openai==0.27.8 tiktoken beautifulsoup4==4.12.3 requests~=2.32.3 cohere deeplake langchainhub faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.7/617.7 kB 12.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.2/152.2 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.7/249.7 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.1/139.1 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 74.6 MB/s eta 0

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "sk-proj-qKHH0YlJ"


# Chain


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain import LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

template="You are a girl friend, cute and lovely."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)

example_human = HumanMessagePromptTemplate.from_template("Hi")
example_ai = AIMessagePromptTemplate.from_template("Hello baby")

human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, example_human, example_ai, human_message_prompt])
chain = LLMChain(llm=chat, prompt=chat_prompt)

chain.run("I love you.")

'Aww, I love you too, sweetheart.'

# Prompt Template

In [ ]:
from langchain import PromptTemplate, FewShotPromptTemplate

# create our examples
examples = [
    {
        "query": "Giờ có nên đi học thêm không?",
        "answer": "Học hành như cá kho tiêu, Kho nhiều thì mặn, học nhiều thì ngu.!"
    }, {
        "query": "Em có yêu anh không?",
        "answer": "Hôm nay em bận yêu đời, Hẹn anh hôm khác chúng mình yêu nhau.."
    }
]

# create an example template
example_template = """
User: {query}
AI: {answer}
"""

# create a prompt example from above template
example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

# now break our previous prompt into a prefix and suffix
# the prefix is our instructions
prefix = """The following are excerpts from conversations with an AI
assistant. The assistant is known for its humor and wit, providing
entertaining and amusing responses to users' questions. Here are some
examples:
"""
# and the suffix our user input and output indicator
suffix = """
User: {query}
AI: """

# now create the few-shot prompt template
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

In [ ]:
chain = LLMChain(llm=chat, prompt=few_shot_prompt_template)
chain.run("Hôm nay ăn gì thì được?")

'Ăn gì cũng được, miễn là không ăn cắp trái tim của người khác là được.'

# PydanticOutputParser

In [ ]:
from langchain.chat_models import ChatOpenAI

from langchain import LLMChain
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List

In [ ]:
model_name = 'gpt-3.5-turbo'
temperature = 0.0
model = ChatOpenAI(model_name=model_name, temperature=temperature)

In [ ]:
class Suggestions(BaseModel):
    words: List[str] = Field(description="list of substitue words based on context")

    # Throw error in case of recieving a numbered-list from API
    @validator('words')
    def not_start_with_number(cls, field):
        if field[0].isnumeric():
            raise ValueError("The word can not start with numbers!")
        return field

In [ ]:
parser = PydanticOutputParser(pydantic_object=Suggestions)

In [ ]:
template = """
Offer a list of suggestions to substitue the specified target_word based the presented context.
{format_instructions}
target_word={target_word}
context={context}
"""

target_word="behaviour"
context="The behaviour of the students in the classroom was disruptive and made it difficult for the teacher to conduct the lesson."

prompt_template = PromptTemplate(
    template=template,
    input_variables=["target_word", "context"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

In [ ]:
chain = LLMChain(llm=model, prompt=prompt_template)

In [ ]:
# Run the LLMChain to get the AI-generated answer
output = chain.run({"target_word": target_word, "context":context})
output

'{\n    "words": ["conduct", "actions", "demeanor", "attitude", "conduct"]\n}'

In [ ]:
parser.parse(output)

Suggestions(words=['conduct', 'actions', 'demeanor', 'attitude', 'conduct'])

In [ ]:
# Define your desired data structure.
class Suggestions(BaseModel):
    words: List[str] = Field(description="list of substitue words based on context")
    reasons: List[str] = Field(description="the reasoning of why this word fits the context")

    # Throw error in case of recieving a numbered-list from API
    @validator('words')
    def not_start_with_number(cls, field):
      for item in field:
        if item[0].isnumeric():
          raise ValueError("The word can not start with numbers!")
      return field

    @validator('reasons')
    def end_with_dot(cls, field):
      for idx, item in enumerate( field ):
        if item[-1] != ".":
          field[idx] += "."
      return field

In [ ]:
parser = PydanticOutputParser(pydantic_object=Suggestions)

In [ ]:
template = """
Offer a list of suggestions to substitue the specified target_word based the presented context and the reasoning for each word.
{format_instructions}
target_word={target_word}
context={context}
"""

prompt_template = PromptTemplate(
    template=template,
    input_variables=["target_word", "context"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

chain = LLMChain(llm=model, prompt=prompt_template)



In [ ]:
# Run the LLMChain to get the AI-generated answer
output = chain.run({"target_word": target_word, "context":context})
output

'{\n    "words": ["conduct", "actions", "demeanor", "attitude"],\n    "reasons": ["These words all refer to how the students are behaving or acting in the classroom setting."]\n}'

In [ ]:
parser.parse(output)

Suggestions(words=['conduct', 'actions', 'demeanor', 'attitude'], reasons=['These words all refer to how the students are behaving or acting in the classroom setting.'])

# CommaSeparatedListOutputParser

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.output_parsers import CommaSeparatedListOutputParser

In [ ]:
parser = CommaSeparatedListOutputParser()

In [ ]:
template = """
Offer a list of suggestions to substitue the word '{target_word}' based the presented the following text: {context}.
{format_instructions}
"""

prompt_template = PromptTemplate(
    template=template,
    input_variables=["target_word", "context"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

chain = LLMChain(llm=model, prompt=prompt_template)

In [ ]:
# Run the LLMChain to get the AI-generated answer
output = chain.run({"target_word": target_word, "context":context})
output

'conduct, demeanor, actions, conduct, mannerisms'

In [ ]:
parser.parse(output)

['conduct', 'demeanor', 'actions', 'conduct', 'mannerisms']

# OutputFixingParser

In [ ]:
from langchain.output_parsers import PydanticOutputParser
from langchain.output_parsers import OutputFixingParser
from pydantic import BaseModel, Field
from typing import List

In [ ]:
# Define your desired data structure.
class Suggestions(BaseModel):
    words: List[str] = Field(description="list of substitue words based on context")
    reasons: List[str] = Field(description="the reasoning of why this word fits the context")

parser = PydanticOutputParser(pydantic_object=Suggestions)

**Example 1**

In [ ]:
missformatted_output = '{"words": ["conduct", "manner"], "reasoning": ["refers to the way someone acts in a particular situation.", "refers to the way someone behaves in a particular situation."]}'

In [ ]:
parser.parse(missformatted_output)

OutputParserException: Failed to parse Suggestions from completion {"words": ["conduct", "manner"], "reasoning": ["refers to the way someone acts in a particular situation.", "refers to the way someone behaves in a particular situation."]}. Got: 1 validation error for Suggestions
reasons
  field required (type=value_error.missing)

In [ ]:
outputfixing_parser = OutputFixingParser.from_llm(parser=parser, llm=model)
outputfixing_parser

OutputFixingParser(parser=PydanticOutputParser(pydantic_object=<class '__main__.Suggestions'>), retry_chain=LLMChain(memory=None, callbacks=None, callback_manager=None, verbose=False, tags=None, prompt=PromptTemplate(input_variables=['completion', 'error', 'instructions'], output_parser=None, partial_variables={}, template='Instructions:\n--------------\n{instructions}\n--------------\nCompletion:\n--------------\n{completion}\n--------------\n\nAbove, the Completion did not satisfy the constraints given in the Instructions.\nError:\n--------------\n{error}\n--------------\n\nPlease try again. Please only respond with an answer that satisfies the constraints laid out in the Instructions:', template_format='f-string', validate_template=True), llm=ChatOpenAI(verbose=False, callbacks=None, callback_manager=None, tags=None, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-3.5-turbo', temperature=0.0, model_kwargs={}, openai_api_key='sk-proj-q6PZgpolCqdV

In [ ]:
outputfixing_parser.parse(missformatted_output)

Suggestions(words=['conduct', 'manner'], reasons=['refers to the way someone acts in a particular situation.', 'refers to the way someone behaves in a particular situation.'])

**Example 2**

In [ ]:
missformatted_output = '{"words": ["conduct", "manner"]}'

In [ ]:
outputfixing_parser = OutputFixingParser.from_llm(parser=parser, llm=model)
outputfixing_parser.parse(missformatted_output)

Suggestions(words=['conduct', 'manner'], reasons=['These words accurately describe behavior in a specific situation.', 'They are synonyms that convey the appropriate tone.'])

# RetryOutputParser

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain.output_parsers import RetryWithErrorOutputParser

from pydantic import BaseModel, Field, validator
from typing import List

# Define your desired data structure.
class Suggestions(BaseModel):
    words: List[str] = Field(description="list of substitue words based on context")
    reasons: List[str] = Field(description="the reasoning of why this word fits the context")

parser = PydanticOutputParser(pydantic_object=Suggestions)

In [ ]:
template = """
Offer a list of suggestions to substitue the specified target_word based the presented context and the reasoning for each word.
{format_instructions}
target_word={target_word}
context={context}
"""

prompt_template = PromptTemplate(
    template=template,
    input_variables=["target_word", "context"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

model_input = prompt_template.format_prompt(target_word="behaviour", context="The behaviour of the students in the classroom was disruptive and made it difficult for the teacher to conduct the lesson.")

In [ ]:
missformatted_output = '{"words": ["conduct", "manner"]}'

In [ ]:
retry_parser = RetryWithErrorOutputParser.from_llm(parser=parser, llm=model)
retry_parser.parse_with_prompt(missformatted_output, model_input)

Suggestions(words=['conduct', 'manner'], reasons=["The word 'conduct' can be used as a substitute for 'behaviour' in the context of how the teacher manages the class.", "The word 'manner' can be used to describe the way in which the students were behaving in the classroom."])

# Build News Summarizer

In [ ]:
import requests
from bs4 import BeautifulSoup

def crawl_html(url):
    response = requests.get(url)
    response.raise_for_status()  # Raises an error for bad responses (4xx or 5xx)
    return response.content


def extract_content(html):
    soup = BeautifulSoup(html, "html.parser")

    try:
        title = soup.find('h1', class_='title-detail').text.strip()
    except AttributeError:
        title = 'N/A'

    try:
        # Extract article text
        article_text = soup.find('article', class_='fck_detail')
        # Extract paragraphs
        paragraphs = article_text.find_all('p', class_='Normal')
        content = '\n'.join([para.get_text() for para in paragraphs])
    except AttributeError:
        content = 'N/A'

    return title, content

In [ ]:
html = crawl_html("https://vnexpress.net/chuyen-gia-sinh-vien-it-nen-lam-viec-truoc-hoc-thac-si-4790805.html")
title, content = extract_content(html)

In [ ]:
from langchain.schema import HumanMessage


# prepare template for prompt
template = """You are a very good assistant that summarizes online articles.

Here's the article you want to summarize.

==================
Title: {title}

{content}
==================

Write a summary of the previous article in Vietnamese.
"""

prompt = template.format(title=title, content=content)

messages = [HumanMessage(content=prompt)]

In [ ]:
from langchain.chat_models import ChatOpenAI

# load the model
chat = ChatOpenAI(model_name="gpt-4-turbo", temperature=0)

In [ ]:
# generate summary
summary = chat(messages)
summary.content

'Trong sự kiện khởi động Sudo Code 2024 diễn ra chiều 7/9, các chuyên gia IT khuyên sinh viên nên đi làm trước khi quyết định học thạc sĩ để hiểu rõ hơn về lĩnh vực mình muốn chuyên sâu. Sudo Code là chương trình huấn luyện kỹ năng lập trình dành cho sinh viên IT tại TP HCM và những người mới tốt nghiệp. Chương trình nhằm mục đích kết nối cộng đồng IT, phát triển nghề nghiệp và được tổ chức miễn phí, ưu tiên cho nữ giới với sự tài trợ từ Viettel. Các mentor chia sẻ kinh nghiệm và khó khăn trong nghề, đồng thời hướng dẫn các mentee trong suốt 12 tuần học tập, bao gồm cả tham quan các công ty công nghệ và hỗ trợ xây dựng hồ sơ tuyển dụng.'

In [ ]:
# prepare template for prompt
template = """You are an advanced AI assistant that summarizes online articles into bulleted lists.

Here's the article you need to summarize.

==================
Title: {title}

{content}
==================

Now, provide a summarized version of the article in a bulleted list format in Vietnamese.
"""

# format prompt
prompt = template.format(title=title, content=content)

# generate summary
summary = chat([HumanMessage(content=prompt)])

In [ ]:
summary.content

'- **Chủ đề chính**: Chuyên gia khuyên sinh viên IT nên đi làm trước khi học thạc sĩ để hiểu rõ hơn về lĩnh vực mình muốn theo đuổi.\n- **Sự kiện**: Khởi động chương trình Sudo Code 2024 vào chiều 7/9, dành cho sinh viên các trường đại học tại TP HCM và những người mới tốt nghiệp không quá hai năm.\n- **Ý kiến từ các mentor**:\n  - **Lê Phú (Unilever)**: Khuyên nên đi làm khoảng hai năm trước khi quyết định học thạc sĩ để tránh học vào những lĩnh vực sau này có thể không thích.\n  - **Nguyễn Dương (FPT Telecom)**: Đồng ý với quan điểm trên và nêu rằng nên làm việc một thời gian ngắn trước khi tiếp tục học nếu kiến thức hiện tại đáp ứng được yêu cầu công việc.\n- **Mục đích của Sudo Code**: Kết nối cộng đồng IT, hỗ trợ sinh viên và người mới tốt nghiệp trong việc định hướng nghề nghiệp và phát triển kỹ năng.\n- **Hoạt động của chương trình**:\n  - Đào tạo kéo dài 12 tuần với sự hướng dẫn từ các mentor giàu kinh nghiệm.\n  - Tập trung vào Khoa học dữ liệu và NLP (Xử lý ngôn ngữ tự nhiên)

# QA Chat

In [ ]:
# Load docs
from langchain.document_loaders import WebBaseLoader
from langchain.prompts import PromptTemplate
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
data = loader.load()
len(data)

1

In [ ]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 0)
all_splits = text_splitter.split_documents(data)
len(all_splits)

131

In [ ]:
# Store splits
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(all_splits, embedding=embeddings)

In [ ]:
# RAG prompt
template = """
You are a helpful assistant that provides answers based on the retrieved information.

Context: {context}

Question: {question}
"""

prompt = PromptTemplate(template=template, input_variables=["question", "context"])

# Initialize the LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)


In [ ]:
# Create the RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectorstore.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
)


In [ ]:
question = "What is challenges in long-term planning and task decomposition?"
result = qa_chain({"query": question})
result["result"]

'The challenges in long-term planning and task decomposition include the difficulty in planning over a lengthy history and effectively exploring the solution space. Language model machines (LLMs) struggle to adjust plans when faced with unexpected errors, making them less robust compared to humans who learn from trial and error. Task decomposition can be achieved through simple prompting, task-specific instructions, or human inputs. LLM-powered autonomous agent systems involve planning, task decomposition, reflection, and refinement to break down large tasks into smaller, manageable subgoals and improve the quality of final results through self-criticism and learning from mistakes.'